**1. Libraries**

In [1]:
from requests_html import HTMLSession
import pandas as pd
import re, os, shutil, sqlite3

**2. Directories**

In [2]:
## Creating directories
path_raw = os.path.join(os.getcwd(), 'Indicator')
if os.path.exists(path_raw) == False: 
    os.mkdir(path_raw)

**3. Connecting**

In [3]:
session = HTMLSession()

In [4]:
url = 'https://www.dane.gov.co/index.php/estadisticas-por-tema/demografia-y-poblacion/estimaciones-del-cambio-demografico'

In [5]:
response = session.get(url)

**3.1 Scrapping**

In [6]:
links = response.html.absolute_links

In [7]:
links

{'http://dane.agenti.com.co/dane/',
 'http://formularios.dane.gov.co/encuestaweb/index.php/234352?newtest=Y',
 'http://geoportal.dane.gov.co/servicios/atlas-estadistico/',
 'http://orfeoott.dane.gov.co/orfeo/login.php',
 'http://www.cancilleria.gov.co',
 'http://www.centroderelevo.gov.co/632/w3-channel.html',
 'http://www.dane.gov.co/',
 'http://www.dane.gov.co/files/dane-para-ninos/index.html',
 'http://www.minagricultura.gov.co',
 'http://www.minambiente.gov.co',
 'http://www.mincit.gov.co',
 'http://www.mincultura.gov.co',
 'http://www.mindefensa.gov.co',
 'http://www.mineducacion.gov.co',
 'http://www.minhacienda.gov.co',
 'http://www.mininterior.gov.co',
 'http://www.minjusticia.gov.co',
 'http://www.minsalud.gov.co',
 'http://www.mintic.gov.co',
 'http://www.mintrabajo.gov.co',
 'http://www.mintransporte.gov.co',
 'http://www.minvivienda.gov.co',
 'http://www.presidencia.gov.co/',
 'http://www.vicepresidencia.gov.co',
 'https://apps.dane.gov.co/crt/certificacion',
 'https://bibli

In [8]:
Links = list()
for link in links:
    if re.search('DCD-PrinInd-crecPobNac-[0-9]{4}-.*.xlsx$', link):
        Links.append(link)
        print(link)
        continue


https://www.dane.gov.co/files/censo2018/cambio-demografico/DCD-PrinInd-crecPobNac-1985-2019.xlsx
https://www.dane.gov.co/files/censo2018/cambio-demografico/DCD-PrinInd-crecPobNac-2020-ActPostCOVID.xlsx


In [9]:
Links

['https://www.dane.gov.co/files/censo2018/cambio-demografico/DCD-PrinInd-crecPobNac-1985-2019.xlsx',
 'https://www.dane.gov.co/files/censo2018/cambio-demografico/DCD-PrinInd-crecPobNac-2020-ActPostCOVID.xlsx']

**3.2 Download and export**

In [10]:
def get_csv(links,csv_name):
    df_all = pd.DataFrame()        
    for web in links:
        print('Reading', web)
        sheet1 = pd.read_excel(web, skiprows=9, converters={'DP':str})
        sheet2 = pd.read_excel(web.replace('crecPobNac','camDemNac'), skiprows=9, converters={'DP':str})
        print('......Cleaning')
        sheet1.columns = sheet1.columns.str.upper()
        sheet2.columns = sheet2.columns.str.upper()
        complete_table = sheet1.merge(sheet2, how='inner', on=['DP', 'DPNOM', 'AÑO'])
        complete_table.columns = complete_table.columns.str.replace(" ", "_")
        complete_table.columns = complete_table.columns.str.replace("\(|\)|\.|\/", "")
        complete_table.columns = complete_table.columns.str.replace("Á", "A")
        complete_table.columns = complete_table.columns.str.replace("Ó", "O")
        complete_table = complete_table.fillna(value = 'NULL')
        complete_table = complete_table.drop(columns=['DPNOM', 'AREA_GEOGRAFICA'], axis=1)
        df_all = pd.concat([df_all, complete_table], ignore_index=True)
        print('......Completed')
    return df_all

In [11]:
df_all = get_csv(Links, 'Indicator\\Indicators.csv')

Reading https://www.dane.gov.co/files/censo2018/cambio-demografico/DCD-PrinInd-crecPobNac-1985-2019.xlsx
......Cleaning
......Completed
Reading https://www.dane.gov.co/files/censo2018/cambio-demografico/DCD-PrinInd-crecPobNac-2020-ActPostCOVID.xlsx


C:\Users\romer\AppData\Local\Temp\ipykernel_11728\3036421880.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  complete_table.columns = complete_table.columns.str.replace("\(|\)|\.|\/", "")


......Cleaning
......Completed


C:\Users\romer\AppData\Local\Temp\ipykernel_11728\3036421880.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  complete_table.columns = complete_table.columns.str.replace("\(|\)|\.|\/", "")


In [12]:
list_str = str()
for col in df_all.columns:
    if col == 'DP':
        list_str = list_str + 'departamento_id' + "   CHAR(2) NULL, "
    else:
        list_str = list_str + col + "  INTEGER NULL,"


In [13]:
con = sqlite3.connect('Colombia_Demographic_data.sqlite', timeout=30)
cur = con.cursor()
cur.executescript('DROP TABLE IF EXISTS Indicadores; CREATE TABLE Indicadores(' + list_str + 'FOREIGN KEY(departamento_id)  REFERENCES Departamento(id));')

In [14]:
con.commit()

In [15]:
list_str = str()
for col in df_all.columns:
    if col == 'DP':
        list_str = list_str + 'departamento_id' + ', '
    else:
        list_str = list_str + col + ', '
list_str = list_str[:-2] + ') VALUES'

In [16]:
insert = 'INSERT INTO Indicadores(' + list_str
for row in range(0,len(df_all)):
    i = 0
    insert += '('
    for feature in df_all.iloc[row]:
        if i == 0:
            insert = insert + "'" + str(feature) + "'" + ','
        else:
            insert = insert + str(feature) + ','
        i = i +1
    insert = insert[:-1] + '),'
    
insert = insert[:-1] + ';'   

In [17]:
cur.executescript(insert)

In [18]:
con.commit()

In [19]:
con.close()